### As usual we setup Weaviate

In [1]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[0], indent=2))

def json_print(data):
    print(json.dumps(data, indent=2))

<class 'list'> 10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [2]:
#Start Weaviate in embedded mode and specify an OpenAI API key

import weaviate
from weaviate import EmbeddedOptions
import os

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]  # Replace this with your actual key
    }
)

embedded weaviate is already listening on port 6666


In [3]:
if client.schema.exists("Question"):
    client.schema.delete_class("Question")

In [4]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  
}

client.schema.create_class(class_obj)

### Create an object by specifying a question, answer and category property

In [6]:
#Create an object

uuid = client.data_object.create(
     data_object={
     'question':"Leonardo da Vinci was born in this country.",
     'answer': "Italy",
     'category': "Culture"
 },
     
     class_name="Question"
    
 )

In [7]:
print(uuid)

6c26a580-7d4b-407d-ae00-2fc691947bcd


### Read object

In [8]:
#Read the object that we just created using it ID
object_uuid = uuid

data_object = client.data_object.get_by_id(object_uuid, class_name="Question")


print(json.dumps(data_object, indent=2))

{
  "class": "Question",
  "creationTimeUnix": 1695848131651,
  "id": "6c26a580-7d4b-407d-ae00-2fc691947bcd",
  "lastUpdateTimeUnix": 1695848131651,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


In [9]:
#Extract the vector for this object

data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
    with_vector=True
)

print(json.dumps(data_object, indent=2))

{
  "class": "Question",
  "creationTimeUnix": 1695848131651,
  "id": "6c26a580-7d4b-407d-ae00-2fc691947bcd",
  "lastUpdateTimeUnix": 1695848131651,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vector": [
    0.022491472,
    -0.013062453,
    -0.0031088893,
    -0.047720812,
    -0.0038829134,
    0.012032554,
    -0.017399546,
    -0.0040556295,
    -0.004519404,
    -0.034671154,
    0.0105100935,
    0.025689917,
    -0.0032112396,
    -0.0072348844,
    0.010337377,
    -0.001473684,
    0.01306885,
    0.0053605954,
    0.03953279,
    -0.00806648,
    0.002354056,
    0.021109743,
    0.011699915,
    -0.01088751,
    0.0051399027,
    0.0018854839,
    0.016516775,
    -0.03216357,
    0.014827995,
    -0.031702995,
    -0.007880971,
    0.0006992602,
    -0.009953563,
    -0.0133950915,
    -0.018742893,
    -0.014431388,
    -0.007682667,
    -0.028095149,
    0.00814964,
    0.015301

### Update object

In [10]:
#Update the object with a more detailed answer

client.data_object.update(
    uuid=object_uuid,
    class_name="Question",
    data_object={
        'answer':"Florence, Italy"
    })

In [11]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
)

print(json.dumps(data_object, indent=2))

{
  "class": "Question",
  "creationTimeUnix": 1695848131651,
  "id": "6c26a580-7d4b-407d-ae00-2fc691947bcd",
  "lastUpdateTimeUnix": 1695848393051,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


### Delete object

In [12]:
#Delete the object using its ID

data_object = client.data_object.get_by_id(object_uuid, class_name="Question")

print(json.dumps(data_object, indent=2))

{
  "class": "Question",
  "creationTimeUnix": 1695848131651,
  "id": "6c26a580-7d4b-407d-ae00-2fc691947bcd",
  "lastUpdateTimeUnix": 1695848393051,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


In [13]:
client.data_object.delete(uuid=object_uuid, class_name="Question")

In [14]:
#Examine that our database is empty

json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 0
          }
        }
      ]
    }
  }
}
